In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import requests
import json
import random
import warnings
import time
import numpy as np
import seaborn as sns
from scipy.stats import chi2_contingency
import plotly.graph_objects as go
import plotly.express as px
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv (r'D:\cars.csv')
df = df.drop('Unnamed: 0', 1)
df= df.rename(columns = {'kilomaters':'kilometers'}, inplace = False)
df

,modle,price,kilometers,stick,engine,year,hand,last_owner,curr_owner,city,car_id
0,"רכב פרטי, אאודי A4","40,000 ₪",220000,ידנית,1984,2005,4,פרטית,פרטית,אזור באר שבע והסביבה,2230947
1,"רכב פרטי, אאודי A4","20,900 ₪",131000,אוטומטית,1984,2006,2,פרטית,פרטית,אזור באר שבע והסביבה,2234559
2,"רכב פרטי, אאודי A6","21,999 ₪",200,אוטומטית,3123,2006,3,פרטית,פרטית,אזור באר שבע והסביבה,2247068
3,"רכב פרטי, אאודי A4","38,000 ₪",195000,אוטומטית,1984,2006,4,פרטית,פרטית,אזור פתח תקוה והסביבה,2254306
4,"רכב פרטי, אאודי A4","5,000 ₪",218073,אוטומטית,1781,2006,3,פרטית,פרטית,אזור רמלה - לוד,2237862
...,...,...,...,...,...,...,...,...,...,...,...
10408,"רכב פרטי, רנו מגאן","102,000 ₪",28000,אוטומטית,1332,2019,1,4/2022,פרטית,אזור עכו - נהריה,2252687
10409,"רכב פרטי, רנו מגאן","98,000 ₪",60000,אוטומטית,1332,2019,1,5/2022,פרטית,אזור חיפה וחוף הכרמל,2252117
10410,"רכב פרטי, רנו מגאן","94,000 ₪",40000,אוטומטית,1332,2019,1,לא,פרטית,אזור ירושלים והסביבה,2137748
10411,"רכב פרטי, רנו מגאן",לא צוין מחיר,35000,אוטומטית,1461,2019,2,פרטית,פרטית,אזור יישובי השומרון,2254705


In [ ]:
duplicate=df2[df2.duplicated(['car_id'])]
duplicate

delete empty cells

In [3]:
df=df.dropna(how='any')
df = df.replace('\n','', regex=True)

delete rows with no price

In [4]:
index_names = df[ df['price'] == 'לא צוין מחיר' ].index
df.drop(index_names, inplace = True)

change all last owners on 1 hand cars to אין

In [5]:
index_names = df [df['hand'] == 1 ].index
index_names

Int64Index([   23,    36,    68,    90,    96,   106,   109,   124,   163,
              167,
            ...
            10400, 10401, 10402, 10403, 10404, 10405, 10406, 10408, 10409,
            10410],
           dtype='int64', length=1717)

In [6]:
df.loc[index_names,'last_owner'] = 'אין'

In [7]:
check_cur=[]
for i in df.curr_owner:
    if 'פרטית' not in i:
        if 'ליסינג 'not in i:
                check_cur.append(i)
print(len(check_cur))

121


In [8]:
df = df[~df.curr_owner.str.contains('|'.join(check_cur))]


In [9]:
#test
for i in df.curr_owner:
    if 'פרטית' not in i:
        if 'ליסינג 'not in i:
                print('false')
print('true')

true


remove all ads that are not a private or leased car on last owner

In [10]:
check_last=[]
for i in df.last_owner:
    if 'פרטית' not in i:
        if 'ליסינג 'not in i:
            if 'אין' not in i:
                check_last.append(i)
print(len(check_last))

687


In [11]:
df = df[~df.last_owner.str.contains('|'.join(check_last))]

In [12]:
#test
for i in df.last_owner:
    if 'פרטית' not in i:
        if 'ליסינג 'not in i:
            if 'אין' not in i:
                print('false')
print('true')

true


change all gears that are not stick to auto

In [13]:
check_stick=[]
for i in df.stick:
    if 'אוטומטית' not in i:
        if 'ידנית 'not in i:
            check_stick.append(i)
print(len(check_stick))

277


In [14]:
df = df.replace(check_stick,'אוטומטית', regex=True)

In [15]:
#test
for i in df.stick:
    if 'אוטומטית' not in i:
        if 'ידנית 'not in i:
            print('false')
print('true')

true


In [16]:
df = df.replace('\xa0₪','', regex=True)
df = df.replace('רכב פרטי,','', regex=True)
df

,modle,price,kilometers,stick,engine,year,hand,last_owner,curr_owner,city,car_id
0,אאודי A4,"40,000",220000,ידנית,1984,2005,4,פרטית,פרטית,אזור באר שבע והסביבה,2230947
1,אאודי A4,"20,900",131000,אוטומטית,1984,2006,2,פרטית,פרטית,אזור באר שבע והסביבה,2234559
2,אאודי A6,"21,999",200,אוטומטית,3123,2006,3,פרטית,פרטית,אזור באר שבע והסביבה,2247068
3,אאודי A4,"38,000",195000,אוטומטית,1984,2006,4,פרטית,פרטית,אזור פתח תקוה והסביבה,2254306
4,אאודי A4,"5,000",218073,אוטומטית,1781,2006,3,פרטית,פרטית,אזור רמלה - לוד,2237862
...,...,...,...,...,...,...,...,...,...,...,...
10406,רנו מגאן,"85,000",92500,אוטומטית,1461,2019,1,אין,פרטית,אזור נס ציונה - רחובות,2254660
10407,רנו מגאן,"95,000",28000,אוטומטית,1461,2020,3,פרטית,פרטית,אזור נס ציונה - רחובות,2230624
10408,רנו מגאן,"102,000",28000,אוטומטית,1332,2019,1,אין,פרטית,אזור עכו - נהריה,2252687
10409,רנו מגאן,"98,000",60000,אוטומטית,1332,2019,1,אין,פרטית,אזור חיפה וחוף הכרמל,2252117


In [17]:
df['price'] = df['price'] .replace(',','', regex=True)


In [18]:
#test years of all ads
for i in df.year:
    if(i<2000 or i>2021):
        print('false')
print( 'true')

true


In [19]:
df.to_csv(r'D:\cars.csv')